In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("aounullahkhan/object-detection-dataset")

print("Path to dataset files:", path)

100%|██████████| 147M/147M [00:00<00:00, 199MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/aounullahkhan/object-detection-dataset/versions/2


In [2]:
import os
len(os.listdir("/root/.cache/kagglehub/datasets/aounullahkhan/object-detection-dataset/versions/2/Images"))

186

In [10]:
!cat "/root/.cache/kagglehub/datasets/aounullahkhan/object-detection-dataset/versions/2/Images Data"

,Filename,Width,Height,Name,xmin,xmax,ymin,ymax
0,Image_1.jpg,1024,1024,car,64,882,113,512
1,Image_10.jpg,726,726,person,149,562,15,484
2,Image_100.jpg,1280,1280,cat,5,1280,2,853
3,Image_101.jpeg,1600,1600,cat,6,1600,15,1519
4,Image_102.jpg,2127,2127,cat,5,2127,1,1894
5,Image_103.jpg,600,600,cat,43,580,40,394
6,Image_104.jpg,1152,1152,cat,1,964,4,833
7,Image_105.jpg,1600,1600,cat,553,1281,33,922
8,Image_106.jpg,800,800,cat,18,775,6,1358
9,Image_107.jpg,500,500,cat,1,463,185,856
10,Image_108.jpg,1200,1200,cat,196,888,81,702
11,Image_109.jpg,4752,4752,cat,323,4150,4,3009
12,Image_11.jpg,1200,1200,car,16,1196,235,637
13,Image_110.jpg,1280,1280,cow,603,1280,93,952
14,Image_110.jpg,1280,1280,cow,1,697,458,960
15,Image_111.jpg,2268,2268,cow,447,1749,640,1561
16,Image_111.jpg,2268,2268,cow,152,2194,182,1488
17,Image_112.jpg,1600,1600,cow,354,1600,56,1200
18,Image_113.JPG,3264,3264,cow,831,1353,568,887
19,Image_113.JPG,3264,3264,cow,2564,2920,605,883
20,Image_113.JPG,3264,3264,cow,727,3001,946

In [3]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

# === CONFIG ===
DATASET_PATH = "/root/.cache/kagglehub/datasets/aounullahkhan/object-detection-dataset/versions/2/Images"
CSV_FILE = "/content/ImagesData.csv"
OUTPUT_DIR = "yolov8_dataset"
SPLIT = [0.7, 0.2, 0.1]  # train, val, test

# === LOAD CSV ===
df = pd.read_csv(CSV_FILE)

# Get unique class names and assign IDs
classes = sorted(df["Name"].unique())
class_to_id = {name: idx for idx, name in enumerate(classes)}

print("Classes:", class_to_id)

# Create YOLO folders
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(OUTPUT_DIR, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, split, "labels"), exist_ok=True)

# === SPLIT FILES ===
image_files = df["Filename"].unique()
train_imgs, test_imgs = train_test_split(image_files, test_size=SPLIT[2], random_state=42)
train_imgs, val_imgs = train_test_split(train_imgs, test_size=SPLIT[1]/(SPLIT[0]+SPLIT[1]), random_state=42)

def get_split(filename):
    if filename in train_imgs:
        return "train"
    elif filename in val_imgs:
        return "val"
    else:
        return "test"

# === CONVERT LABELS AND COPY FILES ===
for img_name in image_files:
    img_df = df[df["Filename"] == img_name]
    img_w = img_df.iloc[0]["Width"]
    img_h = img_df.iloc[0]["Height"]
    split = get_split(img_name)

    label_path = os.path.join(OUTPUT_DIR, split, "labels", img_name.rsplit('.', 1)[0] + ".txt")
    with open(label_path, "w") as f:
        for _, row in img_df.iterrows():
            class_id = class_to_id[row["Name"]]
            x_center = ((row["xmin"] + row["xmax"]) / 2) / img_w
            y_center = ((row["ymin"] + row["ymax"]) / 2) / img_h
            width = (row["xmax"] - row["xmin"]) / img_w
            height = (row["ymax"] - row["ymin"]) / img_h
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # Copy image (fixed path)
    src_img = os.path.join(DATASET_PATH, img_name)
    dst_img = os.path.join(OUTPUT_DIR, split, "images", img_name)

    if os.path.exists(src_img):
        shutil.copy(src_img, dst_img)
    else:
        print(f"⚠️ Image not found: {src_img}")


# === CREATE data.yaml ===
yaml_path = os.path.join(OUTPUT_DIR, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(f"train: {os.path.abspath(os.path.join(OUTPUT_DIR, 'train/images'))}\n")
    f.write(f"val: {os.path.abspath(os.path.join(OUTPUT_DIR, 'val/images'))}\n")
    f.write(f"test: {os.path.abspath(os.path.join(OUTPUT_DIR, 'test/images'))}\n\n")
    f.write(f"nc: {len(classes)}\n")
    f.write(f"names: {classes}\n")

print(f"✅ Dataset ready at: {OUTPUT_DIR}")


Classes: {'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4, 'bus': 5, 'car': 6, 'cat': 7, 'chair': 8, 'cow': 9, 'diningtable': 10, 'dog': 11, 'motorbike': 12, 'person': 13, 'pottedplant': 14, 'sheep': 15, 'sofa': 16, 'train': 17, 'tvmonitor': 18}
✅ Dataset ready at: yolov8_dataset


In [4]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00


In [5]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
model = YOLO("yolov8n.pt")

In [ ]:
model.train(data="/content/yolov8_dataset/data.yaml", epochs=77, imgsz=640, batch=5)

In [9]:
!yolo detect train data=/content/yolov8_dataset/data.yaml model=yolov8n.pt epochs=77 imgsz=640 batch=8 device=0


Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolov8_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=77, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=

In [10]:
import glob
import pandas as pd

# find all results.csv files from YOLO training runs
csv_files = glob.glob("/content/runs/detect/train*/results.csv")

csv_files


['/content/runs/detect/train/results.csv']

In [11]:
best_map = -1
best_run = None

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    # Get the highest mAP50-95 in this run
    run_best_map = df['metrics/mAP50-95(B)'].max()
    if run_best_map > best_map:
        best_map = run_best_map
        best_run = csv_file.split("/results.csv")[0]

print(f"🏆 Best run: {best_run}")
print(f"📈 Best mAP50-95: {best_map:.4f}")


🏆 Best run: /content/runs/detect/train
📈 Best mAP50-95: 0.5410
